#### Install tensorflow package

In [1]:
!pip install tensorflow --upgrade

    100% |████████████████████████████████| 46.5MB 551kB/s ta 0:00:011 8% |██▋                             | 3.7MB 1.9MB/s eta 0:00:23    23% |███████▌                        | 10.9MB 948kB/s eta 0:00:38
Requirement not upgraded as not directly required: absl-py>=0.1.6 in /Users/kishan/anaconda3/lib/python3.6/site-packages (from tensorflow) (0.1.11)
    100% |████████████████████████████████| 3.1MB 2.9MB/s ta 0:00:01
Requirement not upgraded as not directly required: numpy>=1.13.3 in /Users/kishan/anaconda3/lib/python3.6/site-packages (from tensorflow) (1.14.2)
Requirement not upgraded as not directly required: grpcio>=1.8.6 in /Users/kishan/anaconda3/lib/python3.6/site-packages (from tensorflow) (1.10.0)
Requirement not upgraded as not directly required: termcolor>=1.1.0 in /Users/kishan/anaconda3/lib/python3.6/site-packages (from tensorflow) (1.1.0)
Requirement not upgraded as not directly required: wheel>=0.26 in /Users/kishan/anaconda3/lib/python3.6/site-packages (from tensorflow) 

#### Upgrade dask package
This is to prevent a pandas error when working with tensorflow

In [3]:
!pip install dask --upgrade

Requirement already up-to-date: dask in /Users/kishan/anaconda3/lib/python3.6/site-packages (0.17.4)


In [2]:
import numpy as np
import tensorflow as tf

/Users/kishan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Import MNIST data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# Store the MNIST data in /tmp/data
mnist = input_data.read_data_sets("data/mnist_data/", one_hot=True)

Extracting data/mnist_data/train-images-idx3-ubyte.gz
Extracting data/mnist_data/train-labels-idx1-ubyte.gz
Extracting data/mnist_data/t10k-images-idx3-ubyte.gz
Extracting data/mnist_data/t10k-labels-idx1-ubyte.gz


In [6]:
training_digits, training_labels = mnist.train.next_batch(5000)

In [7]:
training_digits

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
training_digits.shape

(5000, 784)

In [9]:
training_labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

#### Effect of one-hot-encoding
The label comprises 10 'columns', one for each digit <br />
The column corresponding to the digit has a value of 1

In [10]:
training_labels.shape

(5000, 10)

In [11]:
test_digits, test_labels = mnist.test.next_batch(200)

### Using numpy 

In [12]:
training_digits.shape

(5000, 784)

In [13]:
reshaped_training_digits = training_digits.reshape(5000, 28, 28)

In [14]:
reshaped_training_digits.shape

(5000, 28, 28)

#### Write a function to crop out edges from a given list of images
The specified number of pixels will be cropped off the edges of each image in the given list

In [15]:
def crop_edges(images, cropx, cropy):
    
    c, x, y = images.shape
    
    startx = x//2 - cropx//2
    starty = y//2 - cropy//2    
    
    return images[:, startx:startx + cropx, starty:starty + cropy]

In [16]:
reshaped_training_digits.shape

(5000, 28, 28)

In [17]:
crop_edges(reshaped_training_digits,24,24).shape

(5000, 24, 24)

In [18]:
cropped_training_digits = crop_edges(reshaped_training_digits,24,24)

In [19]:
cropped_training_digits = cropped_training_digits.reshape(5000,576)

In [20]:
cropped_training_digits.shape

(5000, 576)

In [21]:
test_digits.shape

(200, 784)

In [22]:
reshaped_test_digits = test_digits.reshape(200,28,28)

In [23]:
reshaped_test_digits.shape

(200, 28, 28)

In [24]:
cropped_test_digits = crop_edges(reshaped_test_digits,24,24)

In [25]:
cropped_test_digits.shape

(200, 24, 24)

In [26]:
cropped_test_digits = cropped_test_digits.reshape(200,576)

In [27]:
cropped_test_digits.shape

(200, 576)

### End of numpy reshaping

In [28]:
training_digits_pl = tf.placeholder("float", [None, 784])

In [29]:
cropped_training_digits_pl = tf.placeholder("float", [None, 576])

In [30]:
test_digit_pl = tf.placeholder("float", [784])

In [31]:
cropped_test_digit_pl = tf.placeholder("float", [576])

#### Nearest Neighbor calculation using L1 distance

In [32]:
l1_distance = tf.abs(tf.add(training_digits_pl, 
                            tf.negative(test_digit_pl)))

In [33]:
distance = tf.reduce_sum(l1_distance, axis=1)

In [34]:
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.arg_min(distance, 0)

Instructions for updating:
Use `argmin` instead


In [35]:
accuracy = 0.

In [36]:
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # loop over test data
    for i in range(len(test_digits)):
        # Get nearest neighbor
        nn_index = sess.run(pred, \
        	feed_dict={training_digits_pl: training_digits, 
                       test_digit_pl: test_digits[i, :]})

        # Get nearest neighbor class label and compare it to its true label
        print("Test:", i, 
              "Prediction:", np.argmax(training_labels[nn_index]),
              "True Label:", np.argmax(test_labels[i]))

        # Calculate accuracy
        if np.argmax(training_labels[nn_index]) == np.argmax(test_labels[i]):
            accuracy += 1./len(test_digits)

    print("Done!")
    print("Accuracy:", accuracy)

Test: 0 Prediction: 9 True Label: 9
Test: 1 Prediction: 5 True Label: 5
Test: 2 Prediction: 5 True Label: 5
Test: 3 Prediction: 3 True Label: 3
Test: 4 Prediction: 1 True Label: 1
Test: 5 Prediction: 5 True Label: 5
Test: 6 Prediction: 8 True Label: 8
Test: 7 Prediction: 4 True Label: 4
Test: 8 Prediction: 5 True Label: 5
Test: 9 Prediction: 6 True Label: 6
Test: 10 Prediction: 0 True Label: 0
Test: 11 Prediction: 2 True Label: 2
Test: 12 Prediction: 8 True Label: 8
Test: 13 Prediction: 2 True Label: 2
Test: 14 Prediction: 1 True Label: 1
Test: 15 Prediction: 1 True Label: 1
Test: 16 Prediction: 9 True Label: 9
Test: 17 Prediction: 2 True Label: 2
Test: 18 Prediction: 3 True Label: 3
Test: 19 Prediction: 6 True Label: 6
Test: 20 Prediction: 4 True Label: 4
Test: 21 Prediction: 8 True Label: 8
Test: 22 Prediction: 3 True Label: 3
Test: 23 Prediction: 2 True Label: 2
Test: 24 Prediction: 0 True Label: 0
Test: 25 Prediction: 1 True Label: 1
Test: 26 Prediction: 0 True Label: 0
Test: 27 Pr